In [711]:
import pandas as pd
from pandas import DataFrame, read_csv, Series
import numpy as np
import math
import sklearn.datasets
from sklearn.cluster import KMeans
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [712]:
# получение данных о студентах и их оценках по дисциплинам
import csv
def read_file(filename):
    f = open(filename)
    data = csv.reader(f)
    mentions = dict()
    for line in data:
        student = line[0]
        discipline = line[6]
        type_d = line[7]
        sem_rate = int(line[8])
        bonus = int(line[9])
        exam_rate = int(line[10])
        rate = 0
        if type_d == 'exam':
            if (sem_rate >= 38) & (exam_rate >= 22):
                rate = sem_rate + exam_rate + bonus
        else: 
            if (sem_rate >= 60):
                rate = sem_rate + bonus
        if not student in mentions:
            mentions[student] = dict()
        mentions[student][discipline] = rate
    f.close()
    return mentions

In [713]:
# косинусная мера сходства: насколько два пользователя похожи друг на друга. 
def dist_cosine(vecA, vecB):
    def dot_product(vecA, vecB):
        d = 0.0
        for dim in vecA:
            if dim in vecB: 
                d += vecA[dim]*vecB[dim]
        return d
    a = dot_product(vecA,vecB)
    b = math.sqrt(dot_product(vecA,vecA))
    c = math.sqrt(dot_product(vecB,vecB))
    if (a == 0) or (b == 0) or (c == 0):
        return 0
    else:
        return  (a / (b * c))

In [714]:
def similar_students(studentID, studentsData, nBestStudents = 3):
    matches = [(u, dist_cosine(studentsData[studentID], studentsData[u])) for u in studentsData.keys() if u != studentID]
    bestMatches = sorted(matches, key=lambda q:(q[1], q[0]), reverse=True)[:nBestStudents]
    print("Most correlated with studentID: " + str(studentID))
    for line in bestMatches:
        print("StudentID: " + str(line[0]) + "  Coeff: " + str(line[1])) 
    return [(x[0], x[1]) for x in bestMatches]

In [715]:
students_test = read_file("raw_data32.csv")
#df = DataFrame(students_test)
#df.columns.name = 'studentID'
#df.index.name = 'discipline'    
rec = similar_students('1', students_test, 10)


Most correlated with studentID: 1
StudentID: 9  Coeff: 0.7542152087381664
StudentID: 6  Coeff: 0.7248624347806895
StudentID: 7  Coeff: 0.7194365076928019
StudentID: 5  Coeff: 0.6837763094049188
StudentID: 4  Coeff: 0.5597650673256156
StudentID: 2  Coeff: 0.22271363971105235
StudentID: 3  Coeff: 0.06705017139158977
StudentID: 8  Coeff: 0


In [716]:
# Похожесть дисциплин
def read_subjects(filename):
    f = open(filename)
    data = csv.reader(f)
    subjects = dict()
    for line in data:
        ID = line[0]
        discipline = line[1]
        subjects[ID] = discipline
    f.close()
    return subjects

In [717]:
#sd = read_subjects("disciplines2.csv")

#nltk.download() 

stopwords = set(stopwords.words("russian"))
#print(stopwords)


In [718]:
def review_to_words(review_text):
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё','Е')
    # 0. Перевод английских наименований дисциплин
    # TO DO
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Яa-zA-Z]", " ", review_text) 
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()                             
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stopwords]
    # 4. Формируем текст, объединяя слова через пробел
    return(" ".join(meaningful_words))

In [719]:
d0 = pd.read_csv("disciplines.csv", sep=',', encoding='utf-8')
d0.head(3)

,ID,Name
0,1,Алгоритмы на графах
1,2,Функциональное программирование
2,3,Функциональное программирование


In [738]:
d1 = d0.copy()
d1['Name'] = d1['Name'].apply(lambda s: review_to_words(s))
d = d1.sort_values('Name')
d = d.Name.unique()
d
# кластеризовать уникальные имена, потом смотреть, какому кластеру принадлежит id дисциплины, а затем посмотреть схожие дисциплины

(28,)

In [721]:
#vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 50000, binary = False) 
#vectorized = vectorizer.fit_transform(d1['Name'])
##print(vectorized)

#num_samples, num_features = vectorized.shape
## print((num_samples, num_features)) # 918 дисциплин, 1123-мерный вектор признаков

#vocabulary = vectorizer.get_feature_names()
##print(vocabulary[20])

#num_clusters = 50
#km = KMeans(n_clusters=num_clusters, init='random', n_init=10, verbose=1, random_state=3)
#km.fit(vectorized)


In [722]:
#km.predict(vectorized)
#km.cluster_centers_

In [723]:
##print(km.cluster_centers_)
#print(km.labels_)

In [724]:
#d1['Name'][km.labels_ == 46]

In [725]:
# стемминг
from nltk.stem.snowball import SnowballStemmer
# print(SnowballStemmer("russian").stem("гениально"))  # проверка работы стемминга

In [726]:
rus_stemmer = SnowballStemmer('russian')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (rus_stemmer.stem(w) for w in analyzer(doc))
    
vectorizer = StemmedCountVectorizer(min_df=1, analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = None, binary = False)
#vectorized = vectorizer.fit_transform(d1['Name'])
vectorized = vectorizer.fit_transform(d)

In [727]:
num_samples, num_features = vectorized.shape
vocabulary = vectorizer.get_feature_names()

num_clusters = 350
km = KMeans(n_clusters=num_clusters, init='random', n_init=13, verbose=1, random_state=3)
km.fit(vectorized)

Initialization complete
Iteration  0, inertia 887.000
Iteration  1, inertia 547.805
Iteration  2, inertia 543.602
Converged at iteration 2: center shift 0.000000e+00 within tolerance 4.984511e-07
Initialization complete
Iteration  0, inertia 899.000
Iteration  1, inertia 551.367
Iteration  2, inertia 537.107
Iteration  3, inertia 535.506
Iteration  4, inertia 534.850
Converged at iteration 4: center shift 0.000000e+00 within tolerance 4.984511e-07
Initialization complete
Iteration  0, inertia 866.000
Iteration  1, inertia 545.078
Iteration  2, inertia 531.873
Iteration  3, inertia 531.298
Converged at iteration 3: center shift 0.000000e+00 within tolerance 4.984511e-07
Initialization complete
Iteration  0, inertia 896.000
Iteration  1, inertia 549.863
Iteration  2, inertia 537.188
Iteration  3, inertia 537.066
Converged at iteration 3: center shift 0.000000e+00 within tolerance 4.984511e-07
Initialization complete
Iteration  0, inertia 914.000
Iteration  1, inertia 545.945
Iteration  2

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300,
    n_clusters=350, n_init=13, n_jobs=1, precompute_distances='auto',
    random_state=3, tol=0.0001, verbose=1)

In [728]:
km.predict(vectorized)
km.cluster_centers_

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [729]:
for label in km.labels_: 
    print(label, end = " ")

270 270 238 303 215 43 1 69 60 70 64 239 242 304 326 3 3 91 111 123 347 3 3 3 136 136 81 169 84 186 259 27 147 219 58 342 58 297 273 150 181 58 58 58 335 8 124 211 23 211 210 295 137 155 155 166 14 315 338 16 75 25 23 254 161 208 227 59 25 25 25 25 25 154 154 13 84 211 3 349 86 32 77 192 293 121 285 174 77 256 311 311 190 343 9 249 3 56 232 156 246 31 338 183 345 124 345 50 266 266 138 164 3 23 23 261 3 189 59 117 59 139 10 289 104 318 3 3 142 124 263 319 130 260 23 23 119 10 250 78 59 110 110 110 110 233 325 110 110 110 152 96 41 27 149 76 278 139 72 99 141 153 112 112 60 112 226 226 292 3 294 10 324 35 45 196 196 73 180 180 203 196 0 203 103 129 167 167 44 53 22 307 45 45 53 53 290 116 116 45 175 22 53 53 53 53 137 3 16 268 268 268 124 206 48 268 268 268 268 268 99 39 253 27 27 184 245 73 73 40 207 17 127 27 180 340 257 3 149 284 3 278 322 198 3 218 183 57 154 268 40 333 126 287 224 57 224 57 87 265 302 12 126 308 211 3 122 180 3 124 43 263 151 43 187 182 134 124 160 330 223 160 6 23

In [730]:
#d1['Name'][km.labels_ == 0]
d[km.labels_ == 37]
    # к прошлым данным:
    # 25 кластер "очень плохой" при init = random, num_kluster = 500, n_init = 13, неправильно соотносит предмет "Теория графов"
    # 41 кластер "плохой" при init = k_means++, num_kluster = 500, n_init = 13
    # 62 кластер "плохой" при init = k_means++, num_kluster = 523, n_init = 13
    # хорошое разбиение при init = k_means++, num_kluster = 700, n_init = 13, но очень много "одиночных" дисциплин
    # 23 кластер "плохой" при init = k_means++, num_kluster = 600, n_init = 13
    # 79 кластер "плохой" при init = k_means++, num_kluster = 600, n_init = 15
#d1['Name'][315]
#d1['Name'][km.labels_ == 56].keys() #id дисципли

array(['основы теории вероятностей',
       'теория вероятностей математическая статистика',
       'теория вероятности математическая статистика',
       'теория методика математического образования'], dtype=object)

In [731]:
import scipy as sp

def disciplines_in_one_cluster(from_info, discipline):
    discipline = review_to_words(discipline)
    discipline_vec = vectorizer.transform([discipline])
    discipline_label = km.predict(discipline_vec)
    #return discipline_label #вернуть номер кластера, в котором располагаются схожие дисциплины
    return from_info[km.labels_ == discipline_label]

In [732]:
print("Test 1")
#sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Теория графов")
sim_dis_info = disciplines_in_one_cluster(d, "Теория графов")
print(sim_dis_info)
print()

print("Test 2")
#sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Компьютерные сети")
sim_dis_info = disciplines_in_one_cluster(d, "Компьютерные сети")
print(sim_dis_info)
print()

print("Test 3")
#sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Механика")
sim_dis_info = disciplines_in_one_cluster(d, "Механика")
print(sim_dis_info)
print()

print("Test 4")
#sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Алгоритмы на графах")
sim_dis_info = disciplines_in_one_cluster(d, "Алгоритмы на графах")
print(sim_dis_info)

Test 1
['теория конечных графов']

Test 2
['защита сетях' 'компьютерные сети']

Test 3
['механика разрушений' 'теоретическая механика'
 'теоретическая прикладная механика' 'теоретическая физика']

Test 4
['алгоритмы графах' 'алгоритмы оптимизации графах']


In [733]:
# рекомендации для студента id_student
# информацию о студентах получаем из students_info
# выбираем предметы, основываясь на схожести с n_best_students наиболее похожими студентами
def make_recomendations(id_student, students_info, subjects_info, n_best_students = 3):
    sim_stud = similar_students(id_student, students_info, n_best_students)  
    stud_disciplines = [] 
    disciplines_info = read_subjects("disciplines2.csv")
    for d in students_info[id_student].keys():
        if d not in stud_disciplines:
            #stud_disciplines.append(d)
            stud_disciplines.append(disciplines_info[d])    # дисциплины студента id_student
            sim_dis_info = disciplines_in_one_cluster(subjects_info, disciplines_info[d])
            for sim_dis in sim_dis_info:
                if sim_dis not in stud_disciplines:
                    stud_disciplines.append(sim_dis) # добавление дисциплин из одного кластера. Проблема : обрезанные/исправленные имена дисциплин
    # далее добавление дисциплин "похожих студентов" + дисциплины из одних с ним кластеров
    # далее убираем все дисциплины, принадлежащие одному кластеру, если есть у id_stud плохие оценки по предметам из этого кластера
    # тут возникает новая проблема : а вдруг по предметам из одного кластера у студента разные оценки, например : матан1 - 60 баллов, матан2 - 90 баллов
    return stud_disciplines


# Добавляются неисправленные имена, в конечном списке будет мешанина : исправленные + неисправленные имена
# Посмотреть ф-ию disciplines_in_one_cluster, подправить немного

In [734]:
students_rating = read_file("students_rating.csv")
#print(students_rating)
r = make_recomendations('11', students_rating, d)
r

Most correlated with studentID: 11
StudentID: 5  Coeff: 0.9729210307385605
StudentID: 18  Coeff: 0.9683713392884907
StudentID: 15  Coeff: 0.967386392071021


['Практикум на ЭВМ',
 'практикум решению задач эвм',
 'практикум эвм',
 'практикум эвм гидроаэродинамике',
 'практикум эвм компьютерные сети',
 'практикум эвм численные методы',
 'Математическая логика и теория множеств',
 'cs математическая логика',
 'математическая логика',
 'математическая логика теория множеств',
 'неклассические логики',
 'Математический анализ',
 'математический анализ',
 'математический аппарат базовые модели структурного анализа',
 'Алгебра и геометрия',
 'алгебра геометрия',
 'аналитическая геометрия',
 'геометрия',
 'геометрия многообразий',
 'дифференциальная геометрия теория кривых',
 'дифференциальная геометрия топология',
 'методы алгебраической геометрии',
 'основания геометрии',
 'Основы информатики',
 'основы информатики',
 'теоретические основы информатики',
 'теоретические основы школьного курса информатики',
 'Иностранный язык',
 'иностранный язык',
 'Практикум на ЭВМ',
 'Дискретная математика',
 'актуарная математика',
 'вычислительная математика',